In [13]:
import torch
import numpy as np
import random
import zarr
from skimage.segmentation import relabel_sequential

In [32]:
random.seed(26)
zarr_path = "/mnt/efs/aimbl_2025/student_data/S-DM/Data/zarr_storage/test.zarr"
root = zarr.open (zarr_path)
test_cond = random.choice (list(root.keys()))
test_image = random.choice (list (root[test_cond].keys()))
test_array = np.array (root[test_cond][test_image]["y"], dtype = "int")
print (test_array.shape)
type (test_array)



(11, 2729, 3640)


numpy.ndarray

In [45]:
for i in range(test_array.shape[0]):
    print (test_array[i, :, :].max())

0
0
0
0
89
14
0
0
0
19
2326


In [46]:
collapsed_array = test_array.sum(axis = 0)
print (collapsed_array.shape)
print (collapsed_array.max())

(2729, 3640)
2351


In [50]:
relabelled_array = relabel_sequential(collapsed_array)[0]
print (relabelled_array.max())
2729 //512

2189


5

In [59]:
def crop_tiles (image, crop_size):
    h_tiles = (image.shape[0]//crop_size) + 1
    v_tiles = (image.shape[1]//crop_size) + 1

    new_h_shape = crop_size * h_tiles
    new_v_shape = crop_size * v_tiles

    new_image = np.zeros (shape = (new_h_shape, new_v_shape)) 
    new_image[0:image.shape[0], 0:image.shape[1]] = image
    tiles = []
    for i in range (h_tiles):
        for j in range (v_tiles):
            y_start = i * crop_size
            y_end = (i+1) * crop_size
            x_start = j * crop_size
            x_end = (j+1)* crop_size

            tile = new_image [y_start:y_end , x_start:x_end]
            tiles.append (tile)
    



    return (np.stack (tiles))

crop = crop_tiles (relabelled_array, 512)
print (crop.shape)
print (relabelled_array.shape)





(48, 512, 512)
(2729, 3640)


In [ ]:
def load_data(zarr_path, crop_size):
    root = zarr.open(zarr_path, mode='r')

    # list of diseases
    diseases = root.keys()
    x_list = []
    y_list = []
    for d in diseases:
        for fov in root[d].keys():
            x = root[d][fov]['x']
            y = root[d][fov]['y']

            # Preprocessing
            # collapse masks
            y_collapsed = y.sum(axis = 0)
            y_relabelled = relabel_sequential(y_collapsed)[0]

            # tile x and y

            x_list.append(x_processed)
            y_list.append(y_processed)

    X = np.stack(x_list)
    y = np.stack(y_list)
    return X, y 

In [ ]:

class CenterCropDataset(Dataset):
    def __init__(self, zarr_path, crop_size=512):
        self.root = zarr.open(zarr_path, mode='r')
        self.X = self.root 
        self.Y = self.root['Y']
        self.crop_size = crop_size

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img = self.X[idx]  # shape (C, H, W)
        seg = self.Y[idx]  # shape (C, H, W) or (H, W) if single channel

        # Find centers of objects in segmentation
        centers = get_centers(seg if seg.ndim == 2 else seg[0])
        center = random.choice(centers)

        # Crop both image and segmentation
        img_crop = crop_center(img, center, self.crop_size)
        seg_crop = crop_center(seg, center, self.crop_size)

        return torch.tensor(img_crop, dtype=torch.float32), torch.tensor(seg_crop, dtype=torch.float32)